# Menjalankan QA Tanpa Intermediate Task - Transfer Learning

# Import semua module

In [1]:
#!pip install datasets
#!pip install transformers
#!pip install tensorboard
#!pip install evaluate
#!pip install git+https://github.com/IndoNLP/nusa-crowd.git@release_exp

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [3]:
!pip install --upgrade pip
!pip install -r requirements.txt

  Cloning https://github.com/IndoNLP/nusa-crowd.git (to revision 7748513d20331e72f9969f94f5d43c7f2d4a59a5) to /tmp/pip-install-rvvers_w/nusacrowd_14e056bd964e463490bec2746ed6da9d
  Running command git clone --filter=blob:none --quiet https://github.com/IndoNLP/nusa-crowd.git /tmp/pip-install-rvvers_w/nusacrowd_14e056bd964e463490bec2746ed6da9d
  Running command git rev-parse -q --verify 'sha^7748513d20331e72f9969f94f5d43c7f2d4a59a5'
  Running command git fetch -q https://github.com/IndoNLP/nusa-crowd.git 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Running command git checkout -q 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Resolved https://github.com/IndoNLP/nusa-crowd.git to commit 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Preparing metadata (setup.py) ... done


In [4]:
# Melihat GPU yang tersedia dan penggunaannya.
!nvidia-smi

Sun Mar 26 08:55:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0    56W / 300W |  32476MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   58C    P0   187W / 300W |  32292MiB / 32480MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [5]:
# Memilih GPU yang akan digunakan (contohnya: GPU #7)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [6]:
import transformers
import evaluate
import torch
import operator
import ast
import json
import re
import sys
import contextlib

import numpy as np
import pandas as pd
import torch.nn as nn

from multiprocessing import cpu_count
from evaluate import load
from nusacrowd import NusantaraConfigHelper
from torch.utils.data import DataLoader
from datetime import datetime
from huggingface_hub import notebook_login
from tqdm import tqdm

from datasets import (
    load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict
)
from transformers import (
    BigBirdTokenizerFast,
    BigBirdForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BertForSequenceClassification,
    BertForQuestionAnswering,
    AutoModel, 
    BertTokenizerFast,
    AutoTokenizer, 
    AutoModel, 
    BertTokenizer, 
    BertForPreTraining,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    EvalPrediction,
    AutoModel,
    BertModel
)

# Definisikan hyperparameter

In [7]:
#MODEL_NAME = "indolem/indobert-base-uncased"
#MODEL_NAME = "afaji/fine-tuned-IndoNLI-Translated-with-indobert-base-uncased"
MODEL_NAME = "afaji/fine-tuned-DatasetQAS-TYDI-QA-ID-with-indobert-base-uncased-with-ITTL-without-freeze-LR-1e-05"
#MODEL_NAME = "indobenchmark/indobert-large-p2"
SEED = 42
EPOCH = 1
BATCH_SIZE = 16
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 1e-5
MAX_LENGTH = 400
STRIDE = 100
LOGGING_STEPS = 50
WARMUP_RATIO = 0.06
WEIGHT_DECAY = 0.01
MAXIMUM_SEARCH_ITER =  2
# Untuk mempercepat training, saya ubah SAMPLE menjadi 100.
# Bila mau menggunakan keseluruhan data, gunakan: 
SAMPLE = sys.maxsize
# SAMPLE = 50

# Import dataset QAS

In [8]:
#!pip install git+https://github.com/IndoNLP/nusa-crowd.git

In [9]:
#!pip install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html

In [10]:
#!pip uninstall 

In [11]:
import torch

print(torch.version.cuda)

11.7


In [12]:
#!pip install tensorflow

In [13]:
#import tensorflow as tf
#from tensorflow.python.platform import build_info as build
#print(f"tensorflow version: {tf.__version__}")
#print(f"Cuda Version: {build.build_info['cuda_version']}")
#print(f"Cudnn version: {build.build_info['cudnn_version']}")

In [14]:
#!pip list

In [15]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Mon_Sep_13_19:13:29_PDT_2021
Cuda compilation tools, release 11.5, V11.5.50
Build cuda_11.5.r11.5/compiler.30411180_0


In [16]:
#!pip uninstall torch
#!pip install torch

In [17]:
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2

In [18]:
!nvidia-smi

Sun Mar 26 08:55:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0    57W / 300W |  32476MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   58C    P0   222W / 300W |  32248MiB / 32480MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [19]:
#!pip uninstall -y torch

In [20]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [21]:
!pip install cudatoolkit==11.5

ERROR: Could not find a version that satisfies the requirement cudatoolkit==11.5 (from versions: none)
ERROR: No matching distribution found for cudatoolkit==11.5


In [22]:
!pip install conda

In [23]:
!conda install cudatoolkit=11.5

ERROR: The install method you used for conda--probably either `pip install conda`
or `easy_install conda`--is not compatible with using conda as an application.
If your intention is to install conda as a standalone application, currently
supported install methods include the Anaconda installer and the miniconda
installer.  You can download the miniconda installer from
https://conda.io/miniconda.html.



In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [25]:
torch._C._cuda_getDeviceCount()

0

In [26]:
torch.cuda.is_available()

False

In [27]:
device

device(type='cpu')

In [28]:
print(torch.__version__)

1.13.1+cu117


# Definisikan tokenizer

In [31]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Definisikan fungsi pre-processnya

In [32]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'tydiqa_id' in x.dataset_name)[0].load_dataset()

df_train = pd.DataFrame(data_qas_id['train'])
df_validation = pd.DataFrame(data_qas_id['validation'])

cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in range(len(df_train['context'])):
    answer_start = df_train['context'][i].index(df_train['label'][i])
    answer_end = answer_start + len(df_train['label'][i])
    new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                        'question': df_train["question"][i], 
                                        'answer': {"text": df_train["label"][i], 
                                                   "answer_start": answer_start, 
                                                   "answer_end": answer_end}}, 
                                                   ignore_index=True)

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)    

for i in range(len(df_validation['context'])):
    answer_start = df_validation['context'][i].index(df_validation['label'][i])
    answer_end = answer_start + len(df_validation['label'][i])
    new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                    'question': df_validation["question"][i], 
                                    'answer': {"text": df_validation["label"][i], 
                                               "answer_start": answer_start, 
                                               "answer_end": answer_end}}, 
                                               ignore_index=True)    

train_dataset = Dataset.from_dict(new_df_train)
validation_dataset = Dataset.from_dict(new_df_val)

data_qas_id = DatasetDict({"train": train_dataset, "validation": validation_dataset})

  0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
def rindex(lst, value, operator=operator):
      return len(lst) - operator.indexOf(reversed(lst), value) - 1

def preprocess_function_qa(examples, tokenizer, MAX_LENGTH=MAX_LENGTH, STRIDE=STRIDE, rindex=rindex, operator=operator):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    examples["context"] = [c.lstrip() for c in examples["context"]]

    tokenized_examples = tokenizer(
      examples['question'],
      examples['context'],
      truncation=True,
      max_length = MAX_LENGTH,
      stride=STRIDE,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
      return_tensors='np'
    )

    tokenized_examples['start_positions'] = []
    tokenized_examples['end_positions'] = []

    for seq_idx in range(len(tokenized_examples['input_ids'])):
        seq_ids = tokenized_examples.sequence_ids(seq_idx)
        offset_mappings = tokenized_examples['offset_mapping'][seq_idx]

        cur_example_idx = tokenized_examples['overflow_to_sample_mapping'][seq_idx]

        #answer = examples['answer'][seq_idx][0]
        answer = examples['answer'][cur_example_idx]
        answer = eval(str(answer))
        #answer_text = answer['text'][0]
        answer_start = answer['answer_start']
        #answer_end = answer_start + len(answer_text)
        answer_end = answer['answer_end']

        context_pos_start = seq_ids.index(1)
        context_pos_end = rindex(seq_ids, 1, operator)

        s = e = 0
        if (offset_mappings[context_pos_start][0] <= answer_start and
            offset_mappings[context_pos_end][1] >= answer_end):
          i = context_pos_start
          while offset_mappings[i][0] < answer_start:
            i += 1
          if offset_mappings[i][0] == answer_start:
            s = i
          else:
            s = i - 1

          j = context_pos_end
          while offset_mappings[j][1] > answer_end:
            j -= 1      
          if offset_mappings[j][1] == answer_end:
            e = j
          else:
            e = j + 1

        tokenized_examples['start_positions'].append(s)
        tokenized_examples['end_positions'].append(e)
    return tokenized_examples

# Mulai tokenisasi dan pre-process

In [34]:
tokenized_data_qas_id = data_qas_id.map(
    preprocess_function_qa,
    batched=True,
    remove_columns=data_qas_id['train'].column_names,
    num_proc=1,
    fn_kwargs={'tokenizer': tokenizer, 'MAX_LENGTH': MAX_LENGTH, 'STRIDE': STRIDE, 'rindex': rindex, 'operator': operator}
)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [35]:
tokenized_data_qas_id = tokenized_data_qas_id.remove_columns(["offset_mapping", 
                                            "overflow_to_sample_mapping"])

In [36]:
tokenized_data_qas_id.set_format("torch", columns=["input_ids", "token_type_ids"], output_all_columns=True)

In [37]:
tokenized_data_qas_id_train = Dataset.from_dict(tokenized_data_qas_id["train"][:SAMPLE])
tokenized_data_qas_id_validation = Dataset.from_dict(tokenized_data_qas_id["validation"][:SAMPLE])

# Mendefinisikan argumen (dataops) untuk training nanti

In [38]:
TIME_NOW = str(datetime.now()).replace(":", "-").replace(" ", "_").replace(".", "_")
QA = './results/alur2-idk-mrc'
CHECKPOINT_DIR = f'{QA}-{TIME_NOW}/checkpoint/'
MODEL_DIR = f'{QA}-{TIME_NOW}/model/'
OUTPUT_DIR = f'{QA}-{TIME_NOW}/output/'
ACCURACY_DIR = f'{QA}-{TIME_NOW}/accuracy/'

# Mendefinisikan Training Arguments untuk train

In [39]:
training_args_qa = TrainingArguments(
    
    # Checkpoint
    output_dir=CHECKPOINT_DIR,
    save_strategy='epoch',
    save_total_limit=EPOCH,
    
    # Log
    report_to='tensorboard',
    logging_strategy='steps',
    logging_first_step=True,
    logging_steps=LOGGING_STEPS,
    
    # Train
    num_train_epochs=EPOCH,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    warmup_ratio=WARMUP_RATIO,
    bf16=False,
    dataloader_num_workers=cpu_count(),
    
    # Miscellaneous
    evaluation_strategy='epoch',
    seed=SEED,
)

# Pendefinisian model Question Answering

In [40]:
model_qa = BertForQuestionAnswering.from_pretrained(MODEL_NAME)

In [41]:
model_qa = model_qa.to(device)

# Melakukan pengumpulan data dengan padding

In [42]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Mulai training untuk fine-tune SQUAD diatas IndoBERT

In [43]:
import string
import collections

# # Melakukan evaluasi dari prediksi
def normalize_text(s):
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)
    def white_space_fix(text):
        return " ".join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_f1_prec_rec(pred, gold):
    pred_tokens = normalize_text(pred).split() # True positive + False positive = Untuk precision
    gold_tokens = normalize_text(gold).split() # True positive + False negatives = Untuk recall
    common = collections.Counter(pred_tokens) & collections.Counter(gold_tokens)
    num_same = sum(common.values()) # True positive

    if len(gold_tokens) == 0 or len(pred_tokens) == 0: 
        return int(gold_tokens == pred_tokens)

    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(pred_tokens)
    recall = 1.0 * num_same / len(gold_tokens)
    f1 = (2.0 * precision * recall) / (precision + recall)

    return f1

def compute_metrics(predict_result):
    predictions_idx = np.argmax(predict_result.predictions, axis=2)
    denominator = len(predictions_idx[0])
    label_array = np.asarray(predict_result.label_ids)
    total_correct = 0
    f1_array = []
    precision_array = []
    recall_array = []

    for i in range(len(predict_result.predictions[0])):
        start_pred_idx = predictions_idx[0][i]
        end_pred_idx = predictions_idx[1][i] + 1
        start_gold_idx = label_array[0][i]
        end_gold_idx = label_array[1][i] + 1

        pred_text = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                    [start_pred_idx: end_pred_idx])
        gold_text = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                    [start_gold_idx: end_gold_idx])

        if pred_text == gold_text:
            total_correct += 1

        f1 = compute_f1_prec_rec(pred=pred_text, gold=gold_text)

        f1_array.append(f1)

    exact_match = ((total_correct / denominator) * 100.0)
    final_f1 = np.mean(f1_array) * 100.0

    return {'exact_match': exact_match, 'f1': final_f1}

In [44]:
trainer_qa = Trainer(
    model=model_qa,
    args=training_args_qa,
    #train_dataset=tokenized_data_qas_id_train,
    #eval_dataset=tokenized_data_qas_id_validation,
    tokenizer=tokenizer,
    #data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

In [45]:
#trainer_qa.train()

# Menyimpan model Question Answering

In [46]:
trainer_qa.save_model(MODEL_DIR)

Saving model checkpoint to ./results/alur2-idk-mrc-2023-03-26_08-56-55_690028/model/
Configuration saved in ./results/alur2-idk-mrc-2023-03-26_08-56-55_690028/model/config.json
Model weights saved in ./results/alur2-idk-mrc-2023-03-26_08-56-55_690028/model/pytorch_model.bin
tokenizer config file saved in ./results/alur2-idk-mrc-2023-03-26_08-56-55_690028/model/tokenizer_config.json
Special tokens file saved in ./results/alur2-idk-mrc-2023-03-26_08-56-55_690028/model/special_tokens_map.json


# Melakukan prediksi dari model

In [47]:
predict_result = trainer_qa.predict(tokenized_data_qas_id_validation)
predict_result

***** Running Prediction *****
  Num examples = 573
  Batch size = 8


PredictionOutput(predictions=(array([[-8.351709 , -8.076598 , -8.503143 , ..., -9.282152 , -8.230424 ,
        -8.230472 ],
       [-8.80134  , -7.8416004, -8.878553 , ..., -8.203246 , -8.023994 ,
        -7.9551816],
       [-8.406947 , -7.8792405, -7.5575123, ..., -8.06922  , -8.540083 ,
        -8.542109 ],
       ...,
       [-8.366324 , -7.5886245, -7.8696113, ..., -9.02594  , -8.910944 ,
        -8.83718  ],
       [-7.473398 , -8.006924 , -8.180971 , ..., -8.620615 , -8.630538 ,
        -8.637638 ],
       [-8.5171175, -7.361905 , -8.587641 , ..., -9.035261 , -9.054317 ,
        -9.0132675]], dtype=float32), array([[-8.092256 , -8.070768 , -8.353189 , ..., -8.548285 , -7.212416 ,
        -7.212634 ],
       [-8.535385 , -8.40173  , -8.309284 , ..., -7.9303446, -7.900365 ,
        -7.886168 ],
       [-7.968836 , -7.8870463, -7.9419675, ..., -7.099452 , -8.311144 ,
        -8.295948 ],
       ...,
       [-8.283798 , -7.889635 , -8.555798 , ..., -8.739616 , -8.661089 ,
        -8

In [48]:
os.makedirs(os.path.dirname(OUTPUT_DIR), exist_ok=True)
with open(f'{OUTPUT_DIR}/output.txt', "w") as f:
  f.write(str(predict_result))
  f.close()

# Melakukan evaluasi dari prediksi

In [49]:
metric_result_before_filtering = compute_metrics(predict_result)
metric_result_before_filtering

{'exact_match': 62.129144851657934, 'f1': 76.50970983610755}

In [50]:
os.makedirs(os.path.dirname(ACCURACY_DIR), exist_ok=True)
with open(f'{ACCURACY_DIR}/accuracy.txt', "w") as f:
  f.write(str(metric_result_before_filtering))
  f.close()

## Coba Alur 2

In [51]:
from tqdm import tqdm

def create_qas_dataframe(predict_result=predict_result, index_largest=1):
    predictions_idx = np.argsort(predict_result.predictions, axis=2)[:, :, index_largest * -1]
    #predictions_idx = np.argmax(predict_result.predictions, axis=2)
    label_array = np.asarray(predict_result.label_ids)
    question_decoded = []
    context_decoded = []
    pred_answer_decoded = []
    gold_answer_decoded = []
    
    for i in tqdm(range(len(predict_result.predictions[0]))):
        start_pred_idx = predictions_idx[0][i]
        end_pred_idx = predictions_idx[1][i] + 1
        pred_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_pred_idx: end_pred_idx], skip_special_tokens=True)
        pred_answer_decoded.append(pred_answer)
        
        start_gold_idx = label_array[0][i]
        end_gold_idx = label_array[1][i] + 1
        gold_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_gold_idx: end_gold_idx], skip_special_tokens=True)
        gold_answer_decoded.append(gold_answer)
        
        question = []
        context = []
         
        for j in range(len(tokenized_data_qas_id_validation[i]['token_type_ids'])):
            if tokenized_data_qas_id_validation[i]['token_type_ids'][j] == 0:
                question.append(tokenized_data_qas_id_validation[i]['input_ids'][j])
            else:
                context.append(tokenized_data_qas_id_validation[i]['input_ids'][j])

        question_decoded.append(tokenizer.decode(question, skip_special_tokens=True))
        context_decoded.append(tokenizer.decode(context, skip_special_tokens=True))
    
    qas_df = pd.DataFrame({'Context': context_decoded, 
                           'Question': question_decoded, 
                           'Prediction Answer': pred_answer_decoded,
                          'Gold Answer': gold_answer_decoded})
                      
    return qas_df

In [52]:
qas_df = create_qas_dataframe(predict_result)
qas_df

100%|█████████████████████████████████████████████████████████████████████████████████| 573/573 [03:48<00:00,  2.51it/s]


,Context,Question,Prediction Answer,Gold Answer
0,kolumbus bukanlah orang pertama yang tiba di a...,siapakah yang menemuka benua amerika?,kolumbus,orang - orang viking dari eropa utara
1,kabupaten donggala ( english : donggala regenc...,dimanakah letak donggala?,"provinsi sulawesi tengah, indonesia","provinsi sulawesi tengah, indonesia"
2,awal mula teknik industri dapat ditelusuri dar...,siapa bapak teknik industri?,frederick winslow taylor,frederick winslow taylor
3,penghulu rasyid ( lahir di desa telaga itar ta...,kapan penghulu rasyid meninggal?,15 desember 1861,15 desember 1861
4,samudra pasifik atau lautan teduh ( dari bahas...,seberapa luas kah samudera pasifik?,"179, 7 juta km²","179, 7 juta km²"
...,...,...,...,...
568,pt scooter victory inter part atau yang lebih ...,dimana kantor pusat pt scooter victory inter p...,"bekasi, indonesia","bekasi, indonesia"
569,studi hubungan internasional sebagai teori sud...,kapan teori hubungan internasional diciptakan?,1939,1939
570,musik hip hop atau hip - hop [ 1 ] [ 2 ] atau ...,kapan musik hip hop pertama kali muncul?,1970an,1970an
571,dalam melaksanakan kegiatan belajar - mengajar...,berapa luas smk negeri 1 cikampek?,"28997m², dan jika di tambah dengan luas kampus...",29095m²


In [53]:
question_mark = ['siapa', 'siapakah',
                    'apa', 'apakah', 'adakah',
                    'dimana', 'dimanakah', 'darimanakah',
                    'kapan', 'kapankah',
                    'bagaimana', 'bagaimanakah',
                    'kenapa', 'mengapa',
                    'berapa', 'berapakah', 'seberapa']

## Mencoba cara retrieve model dari HF

In [54]:
!pip install torch torchvision torchaudio

  Using cached torch-2.0.0-cp38-cp38-manylinux1_x86_64.whl (619.9 MB)
  Using cached torchaudio-2.0.1-cp38-cp38-manylinux1_x86_64.whl (4.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.13.1
    Uninstalling torchaudio-0.13.1:
      Successfully uninstalled torchaudio-0.13.1


In [55]:
torch.cuda.is_available()

False

In [56]:
!nvidia-smi

Sun Mar 26 09:02:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0    56W / 300W |  32476MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   59C    P0   245W / 300W |  32292MiB / 32480MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [58]:
from transformers import pipeline

pretrained_name_sc = "afaji/fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased"
pretrained_name_qa = "afaji/fine-tuned-DatasetQAS-TYDI-QA-ID-with-indobert-base-uncased-with-ITTL-without-freeze-LR-1e-05"
tokenizer_kwargs = {'padding': True, 'truncation': True, 'max_length': MAX_LENGTH}

nlp_sc = pipeline(task="text-classification", model=pretrained_name_sc, tokenizer=pretrained_name_sc, 
                   **tokenizer_kwargs)
nlp_qa = pipeline(task="question-answering", model=pretrained_name_qa, tokenizer=pretrained_name_qa, 
                   **tokenizer_kwargs)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--afaji--fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased/snapshots/baf8065c541ffd323cf43d1e93868ccbb20febbd/config.json
Model config BertConfig {
  "_name_or_path": "afaji/fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "positi

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--afaji--fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased/snapshots/baf8065c541ffd323cf43d1e93868ccbb20febbd/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at afaji/fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--afaji--fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased/snapshots/baf8065c541ffd323cf43d1e93868ccbb20febbd/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--afaji--fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased/snapshots/baf8065c541ffd323cf43d1e93868ccbb20febbd/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--afaji--fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased/snapshots/baf8065c541ffd323cf43d1e93868ccbb20febbd/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--afaji--fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased/snapshots/baf8065c541ffd323cf43d1e93868ccbb20febbd/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--afaji-

In [59]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

pretrained_name_tg = "Wikidepia/IndoT5-base-paraphrase"

tokenizer_kwargs = {'truncation': True, 'max_length': MAX_LENGTH}

nlp_tg = pipeline(task="text2text-generation", model=pretrained_name_tg, tokenizer=pretrained_name_tg, 
                  )

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Wikidepia--IndoT5-base-paraphrase/snapshots/5d591dc3aeae0aade0f327a5ebdd0f071c83f567/config.json
Model config T5Config {
  "_name_or_path": "Wikidepia/IndoT5-base-paraphrase",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "use_cache": true,
  "v

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--Wikidepia--IndoT5-base-paraphrase/snapshots/5d591dc3aeae0aade0f327a5ebdd0f071c83f567/pytorch_model.bin
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at Wikidepia/IndoT5-base-paraphrase.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.
Generation config file not found, using a generation config created from the model config.


loading file spiece.model from cache at /root/.cache/huggingface/hub/models--Wikidepia--IndoT5-base-paraphrase/snapshots/5d591dc3aeae0aade0f327a5ebdd0f071c83f567/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--Wikidepia--IndoT5-base-paraphrase/snapshots/5d591dc3aeae0aade0f327a5ebdd0f071c83f567/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--Wikidepia--IndoT5-base-paraphrase/snapshots/5d591dc3aeae0aade0f327a5ebdd0f071c83f567/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Wikidepia--IndoT5-base-paraphrase/snapshots/5d591dc3aeae0aade0f327a5ebdd0f071c83f567/tokenizer_config.json


## Coba buat method evaluasi

In [60]:
def smoothing(question, pred_answer, gold_answer, type, question_mark=question_mark):
    
    if type == 'replace first':
        pred_hypothesis = question.replace('?', '')
        pred_hypothesis = pred_hypothesis.replace(question.split()[0], pred_answer)

        gold_hypothesis = question.replace('?', '')
        gold_hypothesis = gold_hypothesis.replace(question.split()[0], gold_answer)
    
    elif type == 'replace question mark':
        for i in question.split():
            if i in question_mark:
                pred_hypothesis = question.replace('?', '')
                pred_hypothesis = pred_hypothesis.replace(i, pred_answer)

                gold_hypothesis = question.replace('?', '')
                gold_hypothesis = gold_hypothesis.replace(i, gold_answer)
    
    elif type == 'add adalah':
        pred_hypothesis = question.replace('?', '')
        pred_hypothesis = pred_hypothesis.replace(question.split()[0], '')
        pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

        gold_hypothesis = question.replace('?', '')
        gold_hypothesis = gold_hypothesis.replace(question.split()[0], '')
        gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"
    
    elif type == 'just concat answer and question':
        pred_hypothesis = f"{question} {pred_answer}"         
        gold_hypothesis = f"{question} {gold_answer}"
        
    elif type == 'rule based':
        question = question.replace('kah', '')
        for j in question.split():
            if j in question_mark:
                if j == 'siapa' or j == 'siapakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"

                elif j == 'apa' or j == 'apakah' or j == 'adakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"

                elif j == 'dimana' or j == 'dimanakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} di {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} di {gold_answer}"

                elif j == 'darimanakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} dari {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} dari {gold_answer}"

                elif j == 'kapan' or j == 'kapankah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} pada {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} pada {gold_answer}"

                elif j == 'bagaimana' or j == 'bagaimanakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '')
                    pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"

                elif j == 'kenapa' or j == 'mengapa':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, 'alasan').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} adalah karena {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, 'alasan').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah karena {gold_answer}"

                elif j == 'berapa' or j == 'berapakah' or j == 'seberapa': 
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"
                    
    elif type == 'machine generation': 
        pred_hypothesis, gold_hypothesis = smoothing(question, pred_answer, gold_answer, type="rule based")
        pred_hypothesis = nlp_tg(pred_hypothesis)[0]['generated_text']
        gold_hypothesis = nlp_tg(gold_hypothesis)[0]['generated_text']
        
    return pred_hypothesis, gold_hypothesis

In [61]:
pred_hypothesis, gold_hypothesis = smoothing("kenapa messi menang?", "emang jago kali", "tim yang unggul",
                                            type="machine generation")
print(pred_hypothesis)
print(gold_hypothesis)

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



alasan mengapa Messi menang adalah karena dia jago kali
alasan messi menang adalah karena tim yang unggul .


In [62]:
def create_df_for_evaluation(predict_result, type_smoothing, type_qas, MAXIMUM_SEARCH_ITER=MAXIMUM_SEARCH_ITER):
    
    # Ekstrak dari PredictionOutput QAS
    predictions_idx = np.argsort(predict_result.predictions, axis=2)[:, :, 1 * -1]
    label_array = np.asarray(predict_result.label_ids)
    
    question_array = []
    context_array = []
    
    pred_answer_before_filtering_array = []
    pred_answer_after_filtering_array = []
    
    label_before_filtering_array = []
    label_after_filtering_array = []
    
    pred_hypothesis_before_filtering_array = []
    pred_hypothesis_after_filtering_array = []
    
    gold_answer_array = []
    gold_hypothesis_array = []
    
    pred_answer_after_filtering_array_msi_recorded = []
    pred_hypothesis_after_filtering_array_msi_recorded = []
    label_after_filtering_array_msi_recorded = []
    
    # Iterasi ini ditujukan untuk retrieve answer
    for i in tqdm(range(len(predict_result.predictions[0]))):
        
        isFoundBiggest = False
        
        start_pred_idx = predictions_idx[0][i]
        end_pred_idx = predictions_idx[1][i] + 1
        
        start_gold_idx = label_array[0][i]
        end_gold_idx = label_array[1][i] + 1
        
        # Retrieve answer prediksi
        pred_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_pred_idx: end_pred_idx], skip_special_tokens=True)
        
        # Retrieve answer gold
        gold_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_gold_idx: end_gold_idx], skip_special_tokens=True)
        
        question = []
        context = []
        
        # Iterasi ini untuk retrieve question dan context index yang bersangkutan
        for j in range(len(tokenized_data_qas_id_validation[i]['token_type_ids'])):
            
            # Bila token_type_ids-nya 0, maka itu question (sesuai dengan urutan tokenisasi)
            if tokenized_data_qas_id_validation[i]['token_type_ids'][j] == 0:
                question.append(tokenized_data_qas_id_validation[i]['input_ids'][j])
            
            # Bila token_type_ids-nya 1, maka itu context (sesuai dengan urutan tokenisasi)
            else:
                context.append(tokenized_data_qas_id_validation[i]['input_ids'][j])
        
        question_decoded = tokenizer.decode(question, skip_special_tokens=True)
        context_decoded = tokenizer.decode(context, skip_special_tokens=True)
        pred_hypothesis, gold_hypothesis = smoothing(question_decoded, pred_answer, gold_answer, type_smoothing)

        # Cek label dari answer prediksi dan context
        predicted_label = nlp_sc({'text': context_decoded, 
                                  'text_pair': pred_hypothesis}, 
                                 **tokenizer_kwargs)
        
        pred_answer_before_filtering_array.append([pred_answer])
        pred_hypothesis_before_filtering_array.append([pred_hypothesis])
        label_before_filtering_array.append([predicted_label])
        
        # Cek label dari answer prediksi dan context, bila labelnya entailment (atau neutral), maka answernya jadi hasil akhir
        if predicted_label['label'] == 'neutral':
            if type_qas == 'entailment or neutral':
                question_array.append(question_decoded)
                context_array.append(context_decoded)
                pred_answer_after_filtering_array.append([pred_answer])
                gold_answer_array.append(gold_answer)
                pred_hypothesis_after_filtering_array.append([pred_hypothesis])
                gold_hypothesis_array.append(gold_hypothesis)
                label_after_filtering_array.append([predicted_label])

        if predicted_label['label'] == 'entailment':
            if type_qas == 'entailment only' or type_qas == 'entailment or neutral':
                question_array.append(question_decoded)
                context_array.append(context_decoded)
                pred_answer_after_filtering_array.append([pred_answer])
                gold_answer_array.append(gold_answer)
                pred_hypothesis_after_filtering_array.append([pred_hypothesis])
                gold_hypothesis_array.append(gold_hypothesis)
                label_after_filtering_array.append([predicted_label])
            
        # Cek label dari answer prediksi dan context, bila labelnya bukan entailment (atau neutral), 
        # -- maka masuk ke for-loop untuk iterasi ke argmax selanjutnya, dengan menggunakan argsort
        else:
            
            if predicted_label == 'neutral' and type_qas == 'entailment or neutral': continue
            
            # Bila MAXIMUM_SEARCH_ITER dibawah 2, maka continue langsung
            if MAXIMUM_SEARCH_ITER < 2: continue

            # Bila MAXIMUM_SEARCH_ITER diatas 2, maka continue langsung
            else:

                # Bila bukan entailment, loop sebanyak MAXIMUM_SEARCH_ITER kali.
                for index_largest in range(MAXIMUM_SEARCH_ITER - 1):

                    # Cari di index kedua, ketiga, keempat, dan seterusnya
                    predictions_idx_inside_loop = np.argsort(predict_result.predictions, 
                                                             axis=2)[:, :, (index_largest + 2) * -1]

                    start_pred_idx = predictions_idx_inside_loop[0][i]
                    end_pred_idx = predictions_idx_inside_loop[1][i] + 1

                    # Retrieve answer prediksi
                    pred_answer_inside_loop = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                                   [start_pred_idx: end_pred_idx], skip_special_tokens=True)
                    
                    pred_hypothesis_inside_loop, gold_hypothesis = smoothing(
                        question_decoded, pred_answer_inside_loop, gold_answer, type_smoothing)
                    
                    # Cek label dari answer prediksi dan context
                    predicted_label_inside_loop = nlp_sc({'text': context_decoded, 
                                                          'text_pair': pred_hypothesis_inside_loop}
                                                           , **tokenizer_kwargs)
                            
                    pred_answer_after_filtering_array_msi_recorded.append(pred_answer_inside_loop)
                    pred_hypothesis_after_filtering_array_msi_recorded.append(pred_hypothesis_inside_loop)
                    label_after_filtering_array_msi_recorded.append(predicted_label_inside_loop)
                    
                    # Bila label-nya sudah entailment (atau neutral), maka answernya jadi hasil akhir, dan break
                    if type_qas == 'entailment only':
                        if predicted_label_inside_loop['label'] == 'entailment':
                            isFoundBiggest = True
                            question_array.append(question_decoded)
                            context_array.append(context_decoded)
                            gold_answer_array.append(gold_answer)   
                            gold_hypothesis_array.append(gold_hypothesis)
                            
                            pred_answer_after_filtering_array.append(pred_answer_after_filtering_array_msi_recorded)
                            pred_hypothesis_after_filtering_array.append(pred_hypothesis_after_filtering_array_msi_recorded)
                            label_after_filtering_array.append(label_after_filtering_array_msi_recorded)
                            break
                            
                    elif type_qas == 'entailment or neutral':
                        if predicted_label_inside_loop['label'] == 'entailment' or predicted_label_inside_loop['label'] == 'neutral':
                            isFoundBiggest = True
                            question_array.append(question_decoded)
                            context_array.append(context_decoded)
                            gold_answer_array.append(gold_answer)   
                            gold_hypothesis_array.append(gold_hypothesis)
                            
                            pred_answer_after_filtering_array.append(pred_answer_after_filtering_array_msi_recorded)
                            pred_hypothesis_after_filtering_array.append(pred_hypothesis_after_filtering_array_msi_recorded)
                            label_after_filtering_array.append(label_after_filtering_array_msi_recorded)
                            break

                if isFoundBiggest == False:
                    # Bila sampai iterasi terakhir, belum entailment (atau neutral) juga, maka append saja jawaban kosong
                    
                    pred_answer_not_found_biggest = "" # Disini, jawaban kosong
                    
                    question_array.append(question_decoded)
                    context_array.append(context_decoded)
                    
                    pred_hypothesis_not_found_biggest, gold_hypothesis = smoothing(
                        question_decoded, pred_answer_not_found_biggest, gold_answer, type_smoothing)
                    
                    pred_answer_after_filtering_array_msi_recorded.append(pred_answer_not_found_biggest)
                    pred_hypothesis_after_filtering_array_msi_recorded.append(pred_hypothesis_not_found_biggest)
                    label_after_filtering_array_msi_recorded.append(predicted_label_inside_loop)
                    
                    gold_answer_array.append(gold_answer)
                    gold_hypothesis_array.append(gold_hypothesis)
                    
                    pred_answer_after_filtering_array.append(pred_answer_after_filtering_array_msi_recorded)
                    pred_hypothesis_after_filtering_array.append(pred_hypothesis_after_filtering_array_msi_recorded)
                    label_after_filtering_array.append(label_after_filtering_array_msi_recorded)
    
    # Buat DataFrame QAS
    qas_df = pd.DataFrame({'Context': context_array, 
                           'Question': question_array, 
                           
                           'Prediction Answer Before Filtering': pred_answer_before_filtering_array,
                           'Prediction Hypothesis Before Filtering': pred_hypothesis_before_filtering_array,
                           'Label Before Filtering': label_before_filtering_array,
                                 
                           'Prediction Answer After Filtering': pred_answer_after_filtering_array,
                           'Prediction Hypothesis After Filtering': pred_hypothesis_after_filtering_array,
                           'Label After Filtering': label_after_filtering_array,
                          
                           'Gold Answer': gold_answer_array,
                          'Gold Hypothesis': gold_hypothesis_array})
                          
    assert len(predict_result.predictions[0]) == len(qas_df), "Jumlah prediksi berbeda dengan jumlah evaluasi"
    
    # Return DataFrame QAS
    return qas_df

In [63]:
eval_df = create_df_for_evaluation(predict_result, type_smoothing='replace first', type_qas='entailment only')
eval_df

100%|█████████████████████████████████████████████████████████████████████████████████| 573/573 [04:31<00:00,  2.11it/s]


,Context,Question,Prediction Answer Before Filtering,Prediction Hypothesis Before Filtering,Label Before Filtering,Prediction Answer After Filtering,Prediction Hypothesis After Filtering,Label After Filtering,Gold Answer,Gold Hypothesis
0,kolumbus bukanlah orang pertama yang tiba di a...,siapakah yang menemuka benua amerika?,[kolumbus],[kolumbus yang menemuka benua amerika],"[{'label': 'neutral', 'score': 0.4978318512439...","[, , periode 2000 - an bca memperkuat dan meng...","[ yang menemuka benua amerika, yang menemuka ...","[{'label': 'contradiction', 'score': 0.4999576...",orang - orang viking dari eropa utara,orang - orang viking dari eropa utara yang men...
1,kabupaten donggala ( english : donggala regenc...,dimanakah letak donggala?,"[provinsi sulawesi tengah, indonesia]","[provinsi sulawesi tengah, indonesia letak don...","[{'label': 'entailment', 'score': 0.9416803121...","[provinsi sulawesi tengah, indonesia]","[provinsi sulawesi tengah, indonesia letak don...","[{'label': 'entailment', 'score': 0.9416803121...","provinsi sulawesi tengah, indonesia","provinsi sulawesi tengah, indonesia letak dong..."
2,awal mula teknik industri dapat ditelusuri dar...,siapa bapak teknik industri?,[frederick winslow taylor],[frederick winslow taylor bapak teknik industri],"[{'label': 'entailment', 'score': 0.9322418570...",[frederick winslow taylor],[frederick winslow taylor bapak teknik industri],"[{'label': 'entailment', 'score': 0.9322418570...",frederick winslow taylor,frederick winslow taylor bapak teknik industri
3,penghulu rasyid ( lahir di desa telaga itar ta...,kapan penghulu rasyid meninggal?,[15 desember 1861],[15 desember 1861 penghulu rasyid meninggal],"[{'label': 'entailment', 'score': 0.9960013031...",[15 desember 1861],[15 desember 1861 penghulu rasyid meninggal],"[{'label': 'entailment', 'score': 0.9960013031...",15 desember 1861,15 desember 1861 penghulu rasyid meninggal
4,samudra pasifik atau lautan teduh ( dari bahas...,seberapa luas kah samudera pasifik?,"[179, 7 juta km²]","[179, 7 juta km² luas kah samudera pasifik]","[{'label': 'entailment', 'score': 0.9817168116...","[179, 7 juta km²]","[179, 7 juta km² luas kah samudera pasifik]","[{'label': 'entailment', 'score': 0.9817168116...","179, 7 juta km²","179, 7 juta km² luas kah samudera pasifik"
...,...,...,...,...,...,...,...,...,...,...
568,pt scooter victory inter part atau yang lebih ...,dimana kantor pusat pt scooter victory inter p...,"[bekasi, indonesia]","[bekasi, indonesia kantor pusat pt scooter vic...","[{'label': 'entailment', 'score': 0.9985334873...","[bekasi, indonesia]","[bekasi, indonesia kantor pusat pt scooter vic...","[{'label': 'entailment', 'score': 0.9985334873...","bekasi, indonesia","bekasi, indonesia kantor pusat pt scooter vict..."
569,studi hubungan internasional sebagai teori sud...,kapan teori hubungan internasional diciptakan?,[1939],[1939 teori hubungan internasional diciptakan],"[{'label': 'neutral', 'score': 0.9211949110031...","[, , periode 2000 - an bca memperkuat dan meng...","[ yang menemuka benua amerika, yang menemuka ...","[{'label': 'contradiction', 'score': 0.4999576...",1939,1939 teori hubungan internasional diciptakan
570,musik hip hop atau hip - hop [ 1 ] [ 2 ] atau ...,kapan musik hip hop pertama kali muncul?,[1970an],[1970an musik hip hop pertama kali muncul],"[{'label': 'neutral', 'score': 0.6594114303588...","[, , periode 2000 - an bca memperkuat dan meng...","[ yang menemuka benua amerika, yang menemuka ...","[{'label': 'contradiction', 'score': 0.4999576...",1970an,1970an musik hip hop pertama kali muncul
571,dalam melaksanakan kegiatan belajar - mengajar...,berapa luas smk negeri 1 cikampek?,"[28997m², dan jika di tambah dengan luas kampu...","[28997m², dan jika di tambah dengan luas kampu...","[{'label': 'entailment', 'score': 0.9920784831...","[28997m², dan jika di tambah dengan luas kampu...","[28997m², dan jika di tambah dengan luas kampu...","[{'label': 'entailment', 'score': 0.9920784831...",29095m²,29095

In [64]:
eval_df["Prediction Answer Before Filtering"][0][-1]

'kolumbus'

In [65]:
def compute_metrics_from_df(df, type_qas):
    
    denominator = len(df)
    total_correct = 0
    f1_array = []
    
    true_positive_before_filtering = 0
    false_positive_before_filtering = 0
    false_negative_before_filtering = 0
    true_negative_before_filtering = 0
    
    true_positive_after_filtering = 0
    false_positive_after_filtering = 0
    false_negative_after_filtering = 0
    true_negative_after_filtering = 0

    for i in range(len(df)):
        
        pred_answer_before_filtering = df["Prediction Answer Before Filtering"][i][-1]
        pred_answer_after_filtering = df["Prediction Answer After Filtering"][i][-1]
        
        pred_label_before_filtering = df["Label Before Filtering"][i][-1]['label']
        pred_label_after_filtering = df["Label After Filtering"][i][-1]['label']
        
        gold_text = df["Gold Answer"][i]

        if pred_answer_after_filtering == gold_text:
            total_correct += 1

        f1 = compute_f1_prec_rec(pred=pred_answer_after_filtering, gold=gold_text)

        f1_array.append(f1)
        
        # Terprediksi dengan label yang benar, dan hasil answernya benar -> True positive
        # Terprediksi dengan label yang benar, padahal hasil answernya salah -> False positive
        # Terprediksi dengan label yang salah, padahal hasil answernya benar -> False negative
        # Terprediksi dengan label yang salah, dan hasil answernya salah -> True negative
        
        if type_qas == 'entailment only':
        
            if (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering == 'entailment'):
                true_positive_after_filtering += 1
            elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering == 'entailment'):
                false_positive_after_filtering += 1
            elif (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering != 'entailment'):
                false_negative_after_filtering += 1
            elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering != 'entailment'):
                true_negative_after_filtering += 1

            if (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering == 'entailment'):
                true_positive_before_filtering += 1
            elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering == 'entailment'):
                false_positive_before_filtering += 1
            elif (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering != 'entailment'):
                false_negative_before_filtering += 1
            elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering != 'entailment'):
                true_negative_before_filtering += 1
        
        elif type_qas == 'entailment or neutral':
        
            if (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering == 'entailment' 
                                                               or pred_label_after_filtering == 'neutral'):
                true_positive_after_filtering += 1
            elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering == 'entailment' 
                                                                 or pred_label_after_filtering == 'neutral'):
                false_positive_after_filtering += 1
            elif (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering != 'entailment' 
                                                                 and pred_label_after_filtering != 'neutral'):
                false_negative_after_filtering += 1
            elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering != 'entailment' 
                                                                 and pred_label_after_filtering != 'neutral'):
                true_negative_after_filtering += 1

            if (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering == 'entailment' 
                                                                or pred_label_after_filtering == 'neutral'):
                true_positive_before_filtering += 1
            elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering == 'entailment' 
                                                                  or pred_label_after_filtering == 'neutral'):
                false_positive_before_filtering += 1
            elif (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering != 'entailment' 
                                                                  and pred_label_after_filtering != 'neutral'):
                false_negative_before_filtering += 1
            elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering != 'entailment' 
                                                                  and pred_label_after_filtering != 'neutral'):
                true_negative_before_filtering += 1

    exact_match = ((total_correct / denominator) * 100.0)
    final_f1 = np.mean(f1_array) * 100.0
    after_filtering_metric_array = [true_positive_after_filtering, false_positive_after_filtering, 
                          false_negative_after_filtering, true_negative_after_filtering]
    before_filtering_metric_array = [true_positive_before_filtering, false_positive_before_filtering, 
                          false_negative_before_filtering, true_negative_before_filtering]

    return {'exact_match': exact_match, 'f1': final_f1}, after_filtering_metric_array, before_filtering_metric_array

In [66]:
metric_result_after_filtering, after_filtering_metric_array, before_filtering_metric_array = compute_metrics_from_df(
    eval_df, "entailment only")
metric_result_after_filtering

{'exact_match': 47.12041884816754, 'f1': 58.22636147737556}

In [67]:
def convert_to_non_zero(number):
    if number == 0:
        number += sys.float_info.min
    return number

def compute_f1_prec_rec_whole(metric_array):
    accuracy = (metric_array[0] + metric_array[3]) / \
        (metric_array[0] + metric_array[1] + 
         metric_array[2] + metric_array[3])
    
    precision = (metric_array[0]) / (metric_array[0] + metric_array[1])
    
    recall = (metric_array[0]) / (metric_array[0] + metric_array[2])
    
    f1 = (2 * precision * recall) / (precision + recall)
    
    return accuracy, precision, recall, f1

def diff_verbose_metric(metric_result_before, metric_result_after, metric):
    
    percentage = round(((metric_result_after - metric_result_before) / metric_result_before) * 100, 2)
    
    if '&' in metric: vocab = "nilai"
    else: vocab = "metrik"
    
    if metric_result_before ==  metric_result_after:
        print(f"Hasil {vocab} {metric} sebelum filtering NLI SAMA DENGAN metrik setelah filtering NLI")
    elif metric_result_before <  metric_result_after:
        print(f"Hasil {vocab} {metric} setelah filtering NLI mengalami KENAIKAN sebesar: {percentage} %")
    elif metric_result_before >  metric_result_after:
        print(f"Hasil {vocab} {metric} setelah filtering NLI mengalami PENURUNAN sebesar: {-1 * percentage} %")
    
    return percentage

In [68]:
def compare_metrics(metrics_before, metrics_after, 
                    after_filtering_metric_array=after_filtering_metric_array, 
                    before_filtering_metric_array=before_filtering_metric_array):
    
    em_before = metrics_before['exact_match']
    f1_before = metrics_before['f1']

    print("~ METRIK PER TOKEN ~")
    print(f"Skor Exact Match sebelum filtering NLI: {em_before}")
    print(f"Skor F1 sebelum filtering NLI: {f1_before}")
    print()

    em_after = metrics_after['exact_match']
    f1_after = metrics_after['f1']

    print(f"Skor Exact Match setelah filtering NLI: {em_after}")
    print(f"Skor F1 setelah filtering NLI: {f1_after}")
    print()

    em_before = convert_to_non_zero(em_before)
    f1_before = convert_to_non_zero(f1_before)

    em_after = convert_to_non_zero(em_after)
    f1_after = convert_to_non_zero(f1_after)

    print("~ METRIK DENGAN PARAMETER NLI ~")
    print(f"[BEFORE FILTERING] Jawaban benar & label NLI yang sesuai: {before_filtering_metric_array[0]}")
    print(f"[BEFORE FILTERING] Jawaban TIDAK benar & label NLI yang sesuai: {before_filtering_metric_array[1]}")
    print(f"[BEFORE FILTERING] Jawaban benar & label NLI yang TIDAK sesuai: {before_filtering_metric_array[2]}")
    print(f"[BEFORE FILTERING] Jawaban TIDAK benar & label NLI yang TIDAK sesuai: {before_filtering_metric_array[3]}")
    print()

    print(f"[AFTER FILTERING] Jawaban benar & label NLI yang sesuai: {after_filtering_metric_array[0]}")
    print(f"[AFTER FILTERING] Jawaban TIDAK benar & label NLI yang sesuai: {after_filtering_metric_array[1]}")
    print(f"[AFTER FILTERING] Jawaban benar & label NLI yang TIDAK sesuai: {after_filtering_metric_array[2]}")
    print(f"[AFTER FILTERING] Jawaban TIDAK benar & label NLI yang TIDAK sesuai: {after_filtering_metric_array[3]}")
    print()

    print("Metrik di atas, bisa direpresentasikan menjadi:")

    acc_before_whole, prec_before_whole, rec_before_whole, f1_before_whole = compute_f1_prec_rec_whole(
        before_filtering_metric_array)
    acc_after_whole, prec_after_whole, rec_after_whole, f1_after_whole = compute_f1_prec_rec_whole(
        after_filtering_metric_array)

    print(f"[BEFORE FILTERING] Akurasi: {acc_before_whole}")
    print(f"[BEFORE FILTERING] Precision: {prec_before_whole}")
    print(f"[BEFORE FILTERING] Recall: {rec_before_whole}")
    print(f"[BEFORE FILTERING] F1: {f1_before_whole}")
    print()

    print(f"[AFTER FILTERING] Akurasi: {acc_after_whole}")
    print(f"[AFTER FILTERING] Precision: {prec_after_whole}")
    print(f"[AFTER FILTERING] Recall: {rec_after_whole}")
    print(f"[AFTER FILTERING] F1: {f1_after_whole}")
    print()

    print("--- Persentase perubahan hasil metrik ---")
    print("~ METRIK PER TOKEN ~")
    diff_verbose_metric(em_before, em_after, "Exact Match")
    diff_verbose_metric(f1_before, f1_after, "F1")
    print()

    print("~ METRIK DENGAN PARAMETER NLI ~")
    diff_verbose_metric(before_filtering_metric_array[0], after_filtering_metric_array[0], 
                        "Jawaban benar & label NLI yang sesuai")
    diff_verbose_metric(before_filtering_metric_array[1], after_filtering_metric_array[1], 
                        "Jawaban TIDAK benar & label NLI yang sesuai")
    diff_verbose_metric(before_filtering_metric_array[2], after_filtering_metric_array[2], 
                        "Jawaban benar & label NLI yang TIDAK sesuai")
    diff_verbose_metric(before_filtering_metric_array[3], after_filtering_metric_array[3], 
                        "Jawaban TIDAK benar & label NLI yang TIDAK sesuai")
    print()

    print("Metrik di atas, bisa direpresentasikan menjadi:")
    diff_verbose_metric(acc_before_whole, acc_after_whole, "Akurasi")
    diff_verbose_metric(prec_before_whole, prec_after_whole, "Precision")
    diff_verbose_metric(rec_before_whole, rec_after_whole, "Recall")
    diff_verbose_metric(f1_before_whole, f1_after_whole, "F1")
    print()

In [69]:
compare_metrics(metric_result_before_filtering, metric_result_after_filtering)

~ METRIK PER TOKEN ~
Skor Exact Match sebelum filtering NLI: 62.129144851657934
Skor F1 sebelum filtering NLI: 76.50970983610755

Skor Exact Match setelah filtering NLI: 47.12041884816754
Skor F1 setelah filtering NLI: 58.22636147737556

~ METRIK DENGAN PARAMETER NLI ~
[BEFORE FILTERING] Jawaban benar & label NLI yang sesuai: 263
[BEFORE FILTERING] Jawaban TIDAK benar & label NLI yang sesuai: 142
[BEFORE FILTERING] Jawaban benar & label NLI yang TIDAK sesuai: 94
[BEFORE FILTERING] Jawaban TIDAK benar & label NLI yang TIDAK sesuai: 74

[AFTER FILTERING] Jawaban benar & label NLI yang sesuai: 263
[AFTER FILTERING] Jawaban TIDAK benar & label NLI yang sesuai: 142
[AFTER FILTERING] Jawaban benar & label NLI yang TIDAK sesuai: 7
[AFTER FILTERING] Jawaban TIDAK benar & label NLI yang TIDAK sesuai: 161

Metrik di atas, bisa direpresentasikan menjadi:
[BEFORE FILTERING] Akurasi: 0.5881326352530541
[BEFORE FILTERING] Precision: 0.6493827160493827
[BEFORE FILTERING] Recall: 0.7366946778711485
[B

In [70]:
os.makedirs(os.path.dirname(ACCURACY_DIR), exist_ok=True)
with open(f'{ACCURACY_DIR}/metric_comparison_results.txt', "w") as f, contextlib.redirect_stdout(f):
    compare_metrics(metric_result_before_filtering, metric_result_after_filtering)
    f.close()

In [79]:
def new_evaluation(df):
    
    exist_true_answer_label_entailment = 0
    exist_true_answer_label_neutral = 0
    exist_true_answer_label_contradiction = 0
    
    exist_false_answer_label_entailment = 0
    exist_false_answer_label_neutral = 0
    exist_false_answer_label_contradiction = 0
    
    no_exist_true_answer = 0
    no_exist_false_answer = 0
    
    for i in range(len(df)):
        
        pred_answer_before_filtering = df["Prediction Answer Before Filtering"][i][-1]
        pred_answer_after_filtering = df["Prediction Answer After Filtering"][i][-1]
        
        pred_label_before_filtering = df["Label Before Filtering"][i][-1]['label']
        pred_label_after_filtering = df["Label After Filtering"][i][-1]['label']
        
        gold_text = df["Gold Answer"][i]
        
        if (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering == 'entailment') and (pred_answer_after_filtering != ""):
            exist_true_answer_label_entailment += 1
        elif (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering == 'neutral') and (pred_answer_after_filtering != ""):
            exist_true_answer_label_neutral += 1
        elif (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering == 'contradiction') and (pred_answer_after_filtering != ""):
            exist_true_answer_label_contradiction += 1
        
        elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering == 'entailment') and (pred_answer_after_filtering != ""):
            exist_false_answer_label_entailment += 1
        elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering == 'neutral') and (pred_answer_after_filtering != ""):
            exist_false_answer_label_neutral += 1
        elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering == 'contradiction') and (pred_answer_after_filtering != ""):
            exist_false_answer_label_contradiction += 1
        
        elif (pred_answer_after_filtering == gold_text) and (pred_answer_after_filtering == ""):
            no_exist_true_answer += 1
        elif (pred_answer_after_filtering != gold_text) and (pred_answer_after_filtering == ""):
            no_exist_false_answer += 1
    
    print(f"Jawaban benar (answer exist) entailment: {exist_true_answer_label_entailment}")
    print(f"Jawaban benar (answer exist) neutral: {exist_true_answer_label_neutral}")
    print(f"Jawaban benar (answer exist) contradiction: {exist_true_answer_label_contradiction}")
    
    print(f"Jawaban salah (answer exist) entailment: {exist_false_answer_label_entailment}")
    print(f"Jawaban salah (answer exist) neutral: {exist_false_answer_label_neutral}")
    print(f"Jawaban salah (answer exist) contradiction: {exist_false_answer_label_contradiction}")
    
    print(f"Jawaban prediksi no-answer benar: {no_exist_true_answer}")
    print(f"Jawaban prediksi no-answer salah: {no_exist_false_answer}")
    
    return exist_true_answer_label_entailment, \
        exist_true_answer_label_neutral, \
        exist_true_answer_label_contradiction, \
        exist_false_answer_label_entailment, \
        exist_false_answer_label_neutral, \
        exist_false_answer_label_contradiction, \
        no_exist_true_answer, \
        no_exist_false_answer

In [80]:
new_evaluation(eval_df)

Jawaban benar (answer exist) entailment: 263
Jawaban benar (answer exist) neutral: 0
Jawaban benar (answer exist) contradiction: 0
Jawaban salah (answer exist) entailment: 132
Jawaban salah (answer exist) neutral: 0
Jawaban salah (answer exist) contradiction: 0
Jawaban prediksi no-answer benar: 7
Jawaban prediksi no-answer salah: 171


(263, 0, 0, 132, 0, 0, 7, 171)